In [1]:
# =======================[ Load Implementation ]================
%run -i "Insurance_Systems.ipynb"

print("Loaded:", CustomerDC, PolicyDC, CoverageLineDC, PolicyAdmin, ClaimsManager, PolicyModel)


Loaded: <class '__main__.CustomerDC'> <class '__main__.PolicyDC'> <class '__main__.CoverageLineDC'> <class '__main__.PolicyAdmin'> <class '__main__.ClaimsManager'> <class '__main__.PolicyModel'>


In [2]:
# =======================[ Build via Pydantic → Dataclass ]====
from datetime import date

# Customer & Risk (validated by Pydantic)
cust_m = CustomerModel(
    customer_id="CUST-100",
    full_name="Alex Smith",
    risk=RiskProfileModel(prior_claims=0, age=32, location_rating=2),
)

# Coverages (validated; e.g., deductible <= limit)
covs = {
    "Liability": CoverageModel(name="Liability", limit_total=1_000_000, deductible=0,    base_rate=2200),
    "Collision": CoverageModel(name="Collision", limit_total=50_000,    deductible=5_000, base_rate=4200),
    "Theft":     CoverageModel(name="Theft",     limit_total=30_000,    deductible=2_500, base_rate=1800),
}

# Policy (CAR requires Liability + Collision; NCB bounded; VAT default 15%)
pol_m = PolicyModel(
    policy_id="POL-2001",
    product=ProductType.CAR,
    customer=cust_m,
    start_date=date(2025, 1, 1),
    end_date=date(2025, 12, 31),
    coverages=covs,
    ncb_discount=0.10,
    fees_annual=150.0,
    taxes_rate=0.15,
)

# Convert validated model → domain dataclass with logic
policy = pol_m.to_dc()

# Quote & bind
annual_total = PolicyAdmin.quote(policy)
print("Quoted annual total (incl. fees & VAT):", annual_total)
print_premium_breakdown(policy)

PolicyAdmin.bind(policy)
print("Policy status:", policy.status.name)


Quoted annual total (incl. fees & VAT): 7386.45

— Premium Breakdown —
Base (sum coverages):        8200.00
× Risk multiplier:           0.85  → 6970.00
× (1 - NCB 10%):      → 6273.00
+ Endorsements (annual):     0.00  → 6273.00
+ Fees:                      150.00  → 6423.00 (taxable)
× (1 + VAT 15%):   → 7386.45  annual total
Policy status: ACTIVE


In [3]:
# =======================[ Claims (below ded, within limit) ]==
# 1) Collision claim below deductible → payout 0
cl1_m = ClaimModel(claim_id="CLM-001", coverage_name="Collision", gross_loss=3000, loss_date=date(2025, 3, 2))
cl1 = cl1_m.to_dc(policy)
ClaimsManager.approve(cl1); ClaimsManager.settle(cl1)
print_claim_result(cl1, policy.coverages["Collision"])

# 2) Theft claim above deductible but within limit
cl2_m = ClaimModel(claim_id="CLM-002", coverage_name="Theft", gross_loss=12000, loss_date=date(2025, 6, 10))
cl2 = cl2_m.to_dc(policy)
ClaimsManager.approve(cl2); ClaimsManager.settle(cl2)
print_claim_result(cl2, policy.coverages["Theft"])



— Claim CLM-001 — Collision
Gross loss: 3000.00
Status:     SETTLED
Payout:     0.00
Remaining limit (Collision): 50000.00

— Claim CLM-002 — Theft
Gross loss: 12000.00
Status:     SETTLED
Payout:     9500.00
Remaining limit (Theft): 20500.00


In [4]:
# =======================[ Claim exhausting remaining limit ]===
# Make another theft claim that consumes what's left
cl3_m = ClaimModel(claim_id="CLM-003", coverage_name="Theft", gross_loss=25000, loss_date=date(2025, 9, 1))
cl3 = cl3_m.to_dc(policy)
ClaimsManager.approve(cl3); ClaimsManager.settle(cl3)
print_claim_result(cl3, policy.coverages["Theft"])



— Claim CLM-003 — Theft
Gross loss: 25000.00
Status:     SETTLED
Payout:     20500.00
Remaining limit (Theft): 0.00


In [5]:
# =======================[ Endorsement & re-quote ]=============
# Mid-term endorsement increases Collision limit by 20,000 and adds 500 to annual premium
endor_m = EndorsementModel(
    effective_date=date(2025, 7, 1),
    description="Vehicle upgrade — increase Collision limit",
    coverage_changes={"Collision": {"limit_total": 20_000}},
    premium_delta_annual=500.0,
)
endor = endor_m.to_dc()
policy.add_endorsement(endor)

print("\nAfter endorsement:")
print("Collision total limit:", policy.coverages["Collision"].limit_total)
print("Collision remaining:  ", policy.coverages["Collision"].limit_remaining)

new_total = PolicyAdmin.quote(policy)
print("New quoted annual total (incl. fees & VAT):", new_total)
print_premium_breakdown(policy)



After endorsement:
Collision total limit: 70000.0
Collision remaining:   70000.0
New quoted annual total (incl. fees & VAT): 7961.45

— Premium Breakdown —
Base (sum coverages):        8200.00
× Risk multiplier:           0.85  → 6970.00
× (1 - NCB 10%):      → 6273.00
+ Endorsements (annual):     500.00  → 6773.00
+ Fees:                      150.00  → 6923.00 (taxable)
× (1 + VAT 15%):   → 7961.45  annual total


In [6]:
# =======================[ Negative tests (validation) ]=======
# Example of product rule violation: CAR without Liability
try:
    bad_pol = PolicyModel(
        policy_id="POL-FAIL",
        product=ProductType.CAR,
        customer=cust_m,
        start_date=date(2025, 1, 1),
        end_date=date(2025, 12, 31),
        coverages={"Collision": covs["Collision"]},   # missing Liability
    )
except Exception as e:
    print("\nExpected validation error (CAR requires Liability):", e)

# Example of business policy requiring Liability >= 1,000,000
try:
    bus_pol = PolicyModel(
        policy_id="POL-BIZ-1",
        product=ProductType.BUSINESS,
        customer=cust_m,
        start_date=date(2025, 1, 1),
        end_date=date(2025, 12, 31),
        coverages={"Liability": CoverageModel(name="Liability", limit_total=500_000, deductible=0, base_rate=3000)},
    )
except Exception as e:
    print("\nExpected validation error (BUSINESS Liability min):", e)

# Claims blocked on cancelled policy
PolicyAdmin.cancel(policy, reason="Client request")
print("\nPolicy status after cancel:", policy.status.name)
try:
    _ = ClaimModel(claim_id="CLM-004", coverage_name="Collision", gross_loss=5000, loss_date=date(2025, 10, 1)).to_dc(policy)
except Exception as e:
    print("Expected error when claiming on cancelled policy:", e)



Expected validation error (CAR requires Liability): 1 validation error for PolicyModel
__root__
  CAR policies require {'Liability', 'Collision'} coverages. (type=value_error)

Expected validation error (BUSINESS Liability min): 1 validation error for PolicyModel
__root__
  BUSINESS requires Liability limit >= 1,000,000. (type=value_error)

Policy status after cancel: CANCELLED
Expected error when claiming on cancelled policy: Claims allowed only on ACTIVE policies.


In [7]:
# =======================[ HOME Policy Demo ]====================
from datetime import date

# HOME policy with Building & Contents (meets min limits), plus optional Theft
home_covs = {
    "Building": CoverageModel(name="Building", limit_total=1_500_000, deductible=10_000, base_rate=3500),
    "Contents": CoverageModel(name="Contents", limit_total=250_000,   deductible=2_000,  base_rate=1200),
    "Theft":    CoverageModel(name="Theft",    limit_total=100_000,   deductible=2_500,  base_rate=800),
}

home_pol_m = PolicyModel(
    policy_id="POL-HOME-1",
    product=ProductType.HOME,
    customer=cust_m,                         # reuse from earlier demo
    start_date=date(2025, 1, 1),
    end_date=date(2025, 12, 31),
    coverages=home_covs,
    ncb_discount=0.05,                       # 5% no-claim bonus
    taxes_rate=0.15,
    fees_annual=150.0,
)

home_policy = home_pol_m.to_dc()
print("HOME annual total (incl VAT):", PolicyAdmin.quote(home_policy))
print_premium_breakdown(home_policy)
PolicyAdmin.bind(home_policy)
print("HOME status:", home_policy.status.name)

# Burglary claim on Contents: gross 15,000, deductible 2,000 → payout 13,000 (≤ remaining)
cl_home_m = ClaimModel(claim_id="CLM-HOME-1", coverage_name="Contents", gross_loss=15_000, loss_date=date(2025, 5, 10))
cl_home = cl_home_m.to_dc(home_policy)
ClaimsManager.approve(cl_home); ClaimsManager.settle(cl_home)
print_claim_result(cl_home, home_policy.coverages["Contents"])


HOME annual total (incl VAT): 5279.94

— Premium Breakdown —
Base (sum coverages):        5500.00
× Risk multiplier:           0.85  → 4675.00
× (1 - NCB 5%):      → 4441.25
+ Endorsements (annual):     0.00  → 4441.25
+ Fees:                      150.00  → 4591.25 (taxable)
× (1 + VAT 15%):   → 5279.94  annual total
HOME status: ACTIVE

— Claim CLM-HOME-1 — Contents
Gross loss: 15000.00
Status:     SETTLED
Payout:     13000.00
Remaining limit (Contents): 237000.00


In [8]:
# =======================[ LIFE Policy Demo ]====================
# LIFE policy with a single 'Life' coverage (sum assured, zero deductible)
life_covs = {
    "Life": CoverageModel(name="Life", limit_total=500_000, deductible=0, base_rate=2600),
}

life_pol_m = PolicyModel(
    policy_id="POL-LIFE-1",
    product=ProductType.LIFE,
    customer=cust_m,
    start_date=date(2025, 1, 1),
    end_date=date(2025, 12, 31),
    coverages=life_covs,
    ncb_discount=0.00,    # typically not used for life, kept 0
    taxes_rate=0.15,
    fees_annual=150.0,
)

life_policy = life_pol_m.to_dc()
print("LIFE annual total (incl VAT):", PolicyAdmin.quote(life_policy))
print_premium_breakdown(life_policy)
PolicyAdmin.bind(life_policy)
print("LIFE status:", life_policy.status.name)

# Life claim (death benefit): gross equals sum assured → payout up to remaining limit (500,000)
cl_life_m = ClaimModel(claim_id="CLM-LIFE-1", coverage_name="Life", gross_loss=500_000, loss_date=date(2025, 7, 20))
cl_life = cl_life_m.to_dc(life_policy)
ClaimsManager.approve(cl_life); ClaimsManager.settle(cl_life)
print_claim_result(cl_life, life_policy.coverages["Life"])

# A second life claim in the same term will pay 0 because the remaining limit is now 0
try:
    cl_life2_m = ClaimModel(claim_id="CLM-LIFE-2", coverage_name="Life", gross_loss=100_000, loss_date=date(2025, 9, 1))
    cl_life2 = cl_life2_m.to_dc(life_policy)
    ClaimsManager.approve(cl_life2); ClaimsManager.settle(cl_life2)
    print_claim_result(cl_life2, life_policy.coverages["Life"])
except Exception as e:
    print("Second life claim handling:", e)


LIFE annual total (incl VAT): 2714.0

— Premium Breakdown —
Base (sum coverages):        2600.00
× Risk multiplier:           0.85  → 2210.00
× (1 - NCB 0%):      → 2210.00
+ Endorsements (annual):     0.00  → 2210.00
+ Fees:                      150.00  → 2360.00 (taxable)
× (1 + VAT 15%):   → 2714.00  annual total
LIFE status: ACTIVE

— Claim CLM-LIFE-1 — Life
Gross loss: 500000.00
Status:     SETTLED
Payout:     500000.00
Remaining limit (Life): 0.00

— Claim CLM-LIFE-2 — Life
Gross loss: 100000.00
Status:     SETTLED
Payout:     0.00
Remaining limit (Life): 0.00


In [9]:
# =======================[ BUSINESS Policy Demo ]=================
# BUSINESS policy with Liability >= 1,000,000 (required) + Property coverage
biz_covs = {
    "Liability": CoverageModel(name="Liability", limit_total=2_000_000, deductible=0,     base_rate=6000),
    "Property":  CoverageModel(name="Property",  limit_total=1_000_000, deductible=20_000, base_rate=3500),
}

biz_pol_m = PolicyModel(
    policy_id="POL-BIZ-1",
    product=ProductType.BUSINESS,
    customer=cust_m,
    start_date=date(2025, 1, 1),
    end_date=date(2025, 12, 31),
    coverages=biz_covs,
    ncb_discount=0.10,          # 10% NCB
    taxes_rate=0.15,
    fees_annual=250.0,          # higher admin fee for business
)

biz_policy = biz_pol_m.to_dc()
print("BUSINESS annual total (incl VAT):", PolicyAdmin.quote(biz_policy))
print_premium_breakdown(biz_policy)
PolicyAdmin.bind(biz_policy)
print("BUSINESS status:", biz_policy.status.name)

# Property claim: 150,000 loss, deductible 20,000 → payout 130,000 (bounded by remaining)
cl_biz_m = ClaimModel(claim_id="CLM-BIZ-1", coverage_name="Property", gross_loss=150_000, loss_date=date(2025, 2, 14))
cl_biz = cl_biz_m.to_dc(biz_policy)
ClaimsManager.approve(cl_biz); ClaimsManager.settle(cl_biz)
print_claim_result(cl_biz, biz_policy.coverages["Property"])


BUSINESS annual total (incl VAT): 8645.12

— Premium Breakdown —
Base (sum coverages):        9500.00
× Risk multiplier:           0.85  → 8075.00
× (1 - NCB 10%):      → 7267.50
+ Endorsements (annual):     0.00  → 7267.50
+ Fees:                      250.00  → 7517.50 (taxable)
× (1 + VAT 15%):   → 8645.12  annual total
BUSINESS status: ACTIVE

— Claim CLM-BIZ-1 — Property
Gross loss: 150000.00
Status:     SETTLED
Payout:     130000.00
Remaining limit (Property): 870000.00
